In [ ]:
# -*- Bayesian_NN_Train.py -*-
"""
Created Oct 2020

@author: Timothy E H Allen / Alistair M Middleton
"""
#%%

# Import modules
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
import tqdm
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
from sklearn.metrics import mean_absolute_error

tf.enable_v2_behavior()
tfd = tfp.distributions

# Define inputs and variables
receptor = "AR"
input_data_a = "/content/drive/My Drive/Training_Sets/" + receptor + " fingerprint.csv"
input_data_b = "/content/drive/My Drive/Test_Sets/" + receptor + " fingerprint.csv"
rng_1 = 1989
rng_2 = 2020
validation_proportion = 0.25
neurons = 10
hidden_layers = 2
LR = 0.01
epochs = 100
batch_size= 100
model_path = "/content/drive/My Drive/Models/" + receptor + "_quantitative_predictor"

def read_dataset(input_data):
    df = pd.read_csv(input_data)
    X = df[df.columns[0:10000]].values
    Y = df[df.columns[10000]]
    return X, Y

# Load and shuffle data and make training and validation sets

X, Y = read_dataset(input_data_a)
test_x, test_y = read_dataset(input_data_b)
X, Y = shuffle(X, Y, random_state=rng_1)
train_x, validation_x, train_y, validation_y = train_test_split(X, Y, test_size=validation_proportion, random_state=rng_2)

# Note that the kl_loss_weight value here is 1 over the size of the entire dataset, not just the batch size.

kl_loss_weight = 1 / train_x.shape[0]

# Inspect the shape of the training and test data

print("Dimensionality of data:")
print("Train x shape =", train_x.shape)
print("Train y shape =", train_y.shape)
print("Validation x shape =", validation_x.shape)
print("Validation y shape =", validation_y.shape)
print("Test x shape =", test_x.shape)
print("Test y shape =", test_y.shape)


# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1e-5))
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n],
                       scale=1e-5 + tf.nn.softplus(c + t[..., n:])), 
            reinterpreted_batch_ndims=1)),
    ])

# Specify a non-trainable prior
def prior_not_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    pi = .5
    return tf.keras.Sequential([
        tfp.layers.DistributionLambda(lambda t: tfd.Mixture(
            cat=tfd.Categorical(probs=[pi, 1. - pi]),
            components=[tfd.MultivariateNormalDiag(loc=tf.zeros(n), scale_diag=.001 * tf.ones(n)),
                        tfd.MultivariateNormalDiag(loc=tf.zeros(n), scale_diag=1.5 * tf.ones(n))
                        ])
                                      )
    ])


def negloglik(y, rv_y):
    return -rv_y.log_prob(y)

# Specify model architecture

if hidden_layers == 1:
    model_aleatoric_epistemic = tf.keras.Sequential([
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight),
        tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1], scale=1e-5 + tf.math.softplus(1e-5 * t[..., 1:]))) 
        ])
    
elif hidden_layers == 2:
    model_aleatoric_epistemic = tf.keras.Sequential([
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight),
        tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1], scale=1e-5 + tf.math.softplus(1e-5 * t[..., 1:]))) 
        ])
    
elif hidden_layers == 3:
    model_aleatoric_epistemic = tf.keras.Sequential([
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(neurons, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight, activation='relu'),
        tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_not_trainable, kl_weight=kl_loss_weight),
        tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1], scale=1e-5 + tf.math.softplus(1e-5 * t[..., 1:]))) 
        ])
    
else:
    print("Number of hidden layers outside this model scope, please choose 1, 2, or 3")


# Train model
model = model_aleatoric_epistemic
model.compile(optimizer=tf.optimizers.Adam(learning_rate=LR), loss=negloglik, metrics=['mse'])
history = model.fit(train_x, train_y, epochs=epochs,verbose=True, validation_data=(validation_x, validation_y))
model.summary()
model.save(model_path, save_format = "tf")

# Plot history of loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


# Plot history of MSE values
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Calculate test data outputs
y_pred_list = []
for i in tqdm.tqdm(range(500)):
    y_pred = model.predict(test_x)
    y_pred_list.append(y_pred)

y_preds = np.concatenate(y_pred_list, axis=1)
y_mean = np.mean(y_preds, axis=1)
y_sigma = np.std(y_preds, axis=1)

# Plot experimental vs predicted values for test data
plt.figure()
plt.scatter(test_y,y_mean, marker='o')

# Calculate and print mean absolute error values for the training and validation data alongside test result
y_pred_list = []
for i in tqdm.tqdm(range(500)):
    y_pred = model.predict(train_x)
    y_pred_list.append(y_pred)

y_preds = np.concatenate(y_pred_list, axis=1)
y_mean_train = np.mean(y_preds, axis=1)
y_sigma_train = np.std(y_preds, axis=1)

y_pred_list = []
for i in tqdm.tqdm(range(500)):
    y_pred = model.predict(validation_x)
    y_pred_list.append(y_pred)

y_preds = np.concatenate(y_pred_list, axis=1)
y_mean_validation = np.mean(y_preds, axis=1)
y_sigma_validation = np.std(y_preds, axis=1)

print("Training Set MAE:")
print(mean_absolute_error(train_y, y_mean_train))
print("Validation Set MAE:")
print(mean_absolute_error(validation_y, y_mean_validation))
print("Test Set MAE:")
print(mean_absolute_error(test_y, y_mean))


# End the cycle
tf.keras.backend.clear_session()

# Endgame
print("END")